In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver import Keys, ActionChains
import pandas as pd
import requests
from datetime import datetime
from datetime import date
from firebase_admin import credentials, db, firestore, storage
from google.cloud.firestore import GeoPoint
from google.cloud.firestore_v1.base_query import FieldFilter, Or
import firebase_admin
from slugify import slugify

In [2]:
driver = webdriver.Firefox(executable_path=r'/Users/rogerjunior/geckodriver')

/var/folders/41/t4024s9j7lsf7jdsjsqxx5280000gn/T/ipykernel_28688/2890382494.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Firefox(executable_path=r'/Users/rogerjunior/geckodriver')


In [3]:

cred = credentials.Certificate("firebase.json")
firebase_admin.initialize_app(cred, {"storageBucket": "the-queer-spot.appspot.com"})

In [6]:
places_list = pd.read_csv("list-of-places.csv")

def get_all_data(row):

    driver.get(row["googlemaps"])

    if (driver.title == "Before you continue to Google Maps" or driver.title == "Antes de continuar para o Google Maps"):
        driver.implicitly_wait(0.5)
        accept_button = driver.find_elements(by=By.XPATH, value="//*[contains(text(), 'Accept all')] | //*[contains(text(), 'Aceitar tudo')]  ")
        accept_button[1].click()
        driver.implicitly_wait(0.5)
    

    try:
        address = driver.find_element(by=By.XPATH, value="//button[contains(@data-tooltip, 'Copy address')] | //button[contains(@data-tooltip, 'Copiar endereço')]")
        row["readableAddress"] = address.text
    except:
        back_button = driver.find_element(by=By.CLASS_NAME, value="hYBOP")
        back_button.click()
        address = driver.find_element(by=By.XPATH, value="//button[contains(@data-tooltip, 'Copy address')] | //button[contains(@data-tooltip, 'Copiar endereço')]")
        row["readableAddress"] = address.text


    try:
        website = driver.find_element(by=By.XPATH, value="//a[contains(@data-tooltip, 'Open website')], //a[contains(@data-tooltip, 'Abrir Website')]")
        row["website"] = website.get_attribute("href")
    except:
        row["website"] = ""


    try:
        horarios = driver.find_element(by=By.XPATH, value="//img[contains(@src, '//www.gstatic.com/images/icons/material/system_gm/2x/schedule_gm_blue_24dp.png')]")
        horarios.click()
        table = driver.find_element(by=By.CLASS_NAME, value="eK4R0e")
        opening_hours = table.text
        opening_hours.split("\n")

        days_of_the_week = {
            "domingo":"sunday",
            "segunda-feira":"monday",
            "terça-feira":"tuesday",
            "quarta-feira":"wednesday",
            "quinta-feira":"thursday",
            "sexta-feira":"friday",
            "sábado":"saturday"
        }

        raw_opening_hours = opening_hours.split("\n")
        print(raw_opening_hours)
        formated_opening_hours = {}
        for index, x in enumerate(raw_opening_hours):
            if (index % 2) == 0:
                hours_raw = raw_opening_hours[index+1]

                if (hours_raw == "Encerrado"):
                    hours = {
                        "opens": "00:00",
                        "closes": "00:00"
                    }
                else:
                    hours = {
                        "opens": hours_raw.split("–")[0],
                        "closes": hours_raw.split("–")[1]
                    }
                    
                formated_opening_hours[days_of_the_week[x]] = hours

        row["openingHours"] = formated_opening_hours
    except:
        row["openingHours"] = None

    try:
        gallery = driver.find_element(by=By.XPATH, value="//button[contains(@jsaction, 'pane.heroHeaderImage.click')]")
        
        gallery.click()

        driver.implicitly_wait(1)

        photos = driver.find_elements(by=By.CLASS_NAME, value="Uf0tqf")
        photo_urls = []

        for photo in photos:
            ActionChains(driver).scroll(200,200,0,2000).perform() #.move_to_element(photo).perform()
            
            try:
                photo_urls.append(photo.get_attribute('outerHTML').split("&quot;")[1])
            except:
                pass
        row["images"] = photo_urls
    except:
        row["images"] = ""
    
    try:
        coordinates = driver.current_url.split("@")[1].split(",")
        row["coordinates"] = GeoPoint(latitude=float(coordinates[0]), longitude=float(coordinates[1]))
    except:
        row["coordinates"] = ""

    # The API endpoint
    url = "https://api.mapbox.com/search/searchbox/v1/"

    # Adding a payload
    auth = {"access_token": "pk.eyJ1Ijoicm9vb2dlciIsImEiOiJlNDFjYjE5ZjFkZjdkYjc2MmU0Mjc1MjJkOWFjZjcxMCJ9.2HxWJoa5ySs5O6nyytPE5Q", 
               "session_token": datetime.today().strftime('%Y%m%d%H'),}
    payload = {
               "q":row["readableAddress"],
               "country": "pt"
               }

    # A get request to the API
    suggest = requests.get(url + "suggest", params=auth|payload)
    #print(suggest.json()["suggestions"][0]["mapbox_id"])
    retrieve = requests.get(url + "retrieve/" + suggest.json()["suggestions"][0]["mapbox_id"], params=auth)
    row["context"] = retrieve.json()["features"][0]["properties"]["context"]
    
    print(row)


    db = firestore.client()
    row["id"]=slugify(
                row["name"]
            )
    venue = db.collection("venues").document(row["id"]
        )
    
    row["createdAt"] = firestore.SERVER_TIMESTAMP
    row["origin"] = "Google Maps Automation"
    row["public"] = True
    row["vibe"] = {"alone": True}
    row["place"] = {"bar": True}
    row["crowd"] = {"all": True}
    venue.set(row)

    return row

df = pd.DataFrame()

for row in places_list.to_dict(orient='records'):
    print(row['name'])
    df.append(get_all_data(row), ignore_index=True)


Damas
['segunda-feira', 'Encerrado', 'terça-feira', '13:00–01:00', 'quarta-feira', '12:30–01:00', 'quinta-feira', '12:30–01:00', 'sexta-feira', '12:30–04:00', 'sábado', '13:00–04:00', 'domingo', '19:30–01:00']
{'name': 'Damas', 'instagram': 'damas.lisboa', 'website': '', 'googlemaps': 'https://www.google.com/maps/place/Damas/@38.7164236,-9.129447,3a,75y,26.69h,100.77t/data=!3m6!1e1!3m4!1sbhWOhKL105znPF6-mQzAvA!2e0!7i16384!8i8192!4m16!1m9!3m8!1s0xd19338983dc3d31:0x5be1f78dc6bc04c7!2sR.+da+Voz+do+Oper%C3%A1rio+60,+Lisboa!3b1!8m2!3d38.7165014!4d-9.1293831!10e5!16s%2Fg%2F11csd2bzzf!3m5!1s0xd19338983eac577:0x838e19df0563046c!8m2!3d38.7164888!4d-9.129402!16s%2Fg%2F11bwynq_mf?entry=ttu', 'readableAddress': 'R. da Voz do Operário 60, 1170-039 Lisboa', 'openingHours': {'monday': {'opens': '00:00', 'closes': '00:00'}, 'tuesday': {'opens': '13:00', 'closes': '01:00'}, 'wednesday': {'opens': '12:30', 'closes': '01:00'}, 'thursday': {'opens': '12:30', 'closes': '01:00'}, 'friday': {'opens': '12:30'